In [0]:
SILVER_PATH = "wasbs://silver@flightdatastorage.blob.core.windows.net/flight_market/"

GOLD_DIM_DATE_PATH = "wasbs://gold@flightdatastorage.blob.core.windows.net/dim_date/"
GOLD_DIM_AIRPORT_PATH = "wasbs://gold@flightdatastorage.blob.core.windows.net/dim_airport/"
GOLD_DIM_CARRIER_PATH = "wasbs://gold@flightdatastorage.blob.core.windows.net/dim_carrier/"
GOLD_DIM_DISTANCE_GROUP_PATH = "wasbs://gold@flightdatastorage.blob.core.windows.net/dim_distance_group/"

df_silver = spark.read.format("delta").load(SILVER_PATH)

df_dim_date = spark.read.format("delta").load(GOLD_DIM_DATE_PATH)
df_dim_airport = spark.read.format("delta").load(GOLD_DIM_AIRPORT_PATH).filter("is_current = true")
df_dim_carrier = spark.read.format("delta").load(GOLD_DIM_CARRIER_PATH).filter("is_current = true")
df_dim_distance_group = spark.read.format("delta").load(GOLD_DIM_DISTANCE_GROUP_PATH)



current count form silver = 72455618


In [0]:
df_silver.printSchema()

root
 |-- ITIN_ID: string (nullable = true)
 |-- MKT_ID: string (nullable = true)
 |-- MARKET_COUPONS: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- ORIGIN_AIRPORT_ID: string (nullable = true)
 |-- ORIGIN_AIRPORT_SEQ_ID: string (nullable = true)
 |-- ORIGIN_CITY_MARKET_ID: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- ORIGIN_COUNTRY: string (nullable = true)
 |-- ORIGIN_STATE_FIPS: string (nullable = true)
 |-- ORIGIN_STATE_ABR: string (nullable = true)
 |-- ORIGIN_STATE_NM: string (nullable = true)
 |-- ORIGIN_WAC: string (nullable = true)
 |-- DEST_AIRPORT_ID: string (nullable = true)
 |-- DEST_AIRPORT_SEQ_ID: string (nullable = true)
 |-- DEST_CITY_MARKET_ID: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- DEST_COUNTRY: string (nullable = true)
 |-- DEST_STATE_FIPS: string (nullable = true)
 |-- DEST_STATE_ABR: string (nullable = true)
 |-- DEST_STATE_NM: string (nullable = true)
 |-

###### Prepare Fact Base Dataset

In [0]:
from pyspark.sql.functions import col

df_fact_base = (
    df_silver
    .select(
        col("ITIN_ID").alias("itin_id"),
        col("MKT_ID").alias("mkt_id"),
        col("MARKET_COUPONS").alias("market_coupons"),
        col("BULK_FARE").alias("bulk_fare"),

        col("year"),
        col("quarter"),

        col("ORIGIN_AIRPORT_SEQ_ID").alias("origin_airport_seq_id"),
        col("DEST_AIRPORT_SEQ_ID").alias("dest_airport_seq_id"),

        col("REPORTING_CARRIER").alias("reporting_carrier"),
        col("TICKET_CARRIER").alias("ticketing_carrier"),
        col("OPERATING_CARRIER").alias("operating_carrier"),

        col("DISTANCE_GROUP").alias("distance_group_code"),

        col("passenger_cnt"),
        col("market_fare_usd"),
        col("market_distance_miles"),
        col("market_miles_flown"),
        col("nonstop_miles"),
        col("source_year"),
        col("source_quarter")
    )
)


In [0]:
df_fact_base= df_fact_base.dropDuplicates()
df_dim_date = df_dim_date.dropDuplicates()
df_dim_airport = df_dim_airport.dropDuplicates()
df_dim_carrier = df_dim_carrier.dropDuplicates()
df_dim_distance_group = df_dim_distance_group.dropDuplicates()


In [0]:
df_fact_base.count()

DEBUG:ThreadMonitor:Logging python thread stack frames for MainThread and py4j threads:
DEBUG:ThreadMonitor:Logging Thread-56 (run) stack frames:
  File "/usr/lib/python3.12/threading.py", line 1030, in _bootstrap
    self._bootstrap_inner()
  File "/usr/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/databricks/python/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 521, in run
    self.wait_for_commands()
  File "/databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 593, in wait_for_commands
    command = smart_decode(self.stream.readline())[:-1]
  File "/usr/lib/python3.12/socket.py", line 707, in readinto
    return self._sock.recv_into(b)

DEBUG:ThreadMonitor:Logging Th

39876323

In [0]:
df_dim_airport.count()

461

In [0]:
df_dim_airport.count()

461

In [0]:
df_dim_carrier.count()

62

In [0]:
df_dim_distance_group.count()

24

###### Join Date Dimension

In [0]:
df_fact.count()

DEBUG:ThreadMonitor:Logging python thread stack frames for MainThread and py4j threads:
DEBUG:ThreadMonitor:Logging Thread-57 (run) stack frames:
  File "/usr/lib/python3.12/threading.py", line 1030, in _bootstrap
    self._bootstrap_inner()
  File "/usr/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/databricks/python/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 521, in run
    self.wait_for_commands()
  File "/databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 593, in wait_for_commands
    command = smart_decode(self.stream.readline())[:-1]
  File "/usr/lib/python3.12/socket.py", line 707, in readinto
    return self._sock.recv_into(b)

DEBUG:ThreadMonitor:Logging Th

3622449206

###### Join Airport Dimensions

In [0]:
df_fact = (
    df_fact_base
    .join(
        df_dim_airport.select(
            col("airport_seq_id").alias("origin_airport_seq_id"),
            col("airport_key").alias("origin_airport_key")
        ),
        on="origin_airport_seq_id",
        how="left"
    )
    .join(
        df_dim_airport.select(
            col("airport_seq_id").alias("dest_airport_seq_id"),
            col("airport_key").alias("dest_airport_key")
        ),
        on="dest_airport_seq_id",
        how="left"
    )
)


In [0]:
df_fact.count()

DEBUG:ThreadMonitor:Logging python thread stack frames for MainThread and py4j threads:
DEBUG:ThreadMonitor:Logging Thread-63 (run) stack frames:
  File "/usr/lib/python3.12/threading.py", line 1030, in _bootstrap
    self._bootstrap_inner()
  File "/usr/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/databricks/python/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 521, in run
    self.wait_for_commands()
  File "/databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 593, in wait_for_commands
    command = smart_decode(self.stream.readline())[:-1]
  File "/usr/lib/python3.12/socket.py", line 707, in readinto
    return self._sock.recv_into(b)

DEBUG:ThreadMonitor:Logging Th

39876323

###### Join Carrier Dimensions

In [0]:
df_fact = (
    df_fact
    .join(
        df_dim_carrier.select(
            col("carrier_code").alias("reporting_carrier"),
            col("carrier_key").alias("reporting_carrier_key")
        ),
        on="reporting_carrier",
        how="left"
    )
    .join(
        df_dim_carrier.select(
            col("carrier_code").alias("ticketing_carrier"),
            col("carrier_key").alias("ticketing_carrier_key")
        ),
        on="ticketing_carrier",
        how="left"
    )
    .join(
        df_dim_carrier.select(
            col("carrier_code").alias("operating_carrier"),
            col("carrier_key").alias("operating_carrier_key")
        ),
        on="operating_carrier",
        how="left"
    )
)


###### Join Distance Group Dimension

In [0]:
df_fact = (
    df_fact
    .join(
        df_dim_distance_group.select(
            col("distance_group_code"),
            col("distance_group_key")
        ),
        on="distance_group_code",
        how="left"
    )
)


###### Finalize Fact Dataset

In [0]:
df_fact_airline_tickets = (
    df_fact
    .select(
        "itin_id",
        "mkt_id",
        "market_coupons",
        "bulk_fare",
        
        "origin_airport_key",
        "dest_airport_key",
        "reporting_carrier_key",
        "ticketing_carrier_key",
        "operating_carrier_key",
        "distance_group_key",

        "passenger_cnt",
        "market_fare_usd",
        "market_distance_miles",
        "market_miles_flown",
        "nonstop_miles",
        "source_year",
        "source_quarter"
    )
)


In [0]:
df_fact_airline_tickets.count()

DEBUG:ThreadMonitor:Logging python thread stack frames for MainThread and py4j threads:
DEBUG:ThreadMonitor:Logging Thread-64 (run) stack frames:
  File "/usr/lib/python3.12/threading.py", line 1030, in _bootstrap
    self._bootstrap_inner()
  File "/usr/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/databricks/python/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 521, in run
    self.wait_for_commands()
  File "/databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 593, in wait_for_commands
    command = smart_decode(self.stream.readline())[:-1]
  File "/usr/lib/python3.12/socket.py", line 707, in readinto
    return self._sock.recv_into(b)

DEBUG:ThreadMonitor:Logging Th

39876323

In [0]:
GOLD_FACT_AIRLINE_TICKETS_PATH = (
    "wasbs://gold@flightdatastorage.blob.core.windows.net/fact_airline_tickets/"
)

In [0]:
rowcount = df_qtr = df_fact_airline_tickets.filter(
    (col("source_year") == "2024") &
    (col("source_quarter") == "02")
).count()

In [0]:
print(rowcount)

8533978


In [0]:
df_qtr = df_fact_airline_tickets.filter(
    (col("source_year") == "2024") &
    (col("source_quarter") == "02")
)

(
    df_qtr
    .write
    .format("delta")
    .mode("append")
    .partitionBy("source_year", "source_quarter")
    .save(GOLD_FACT_AIRLINE_TICKETS_PATH)
)

DEBUG:ThreadMonitor:Logging python thread stack frames for MainThread and py4j threads:
DEBUG:ThreadMonitor:Logging Thread-65 (run) stack frames:
  File "/usr/lib/python3.12/threading.py", line 1030, in _bootstrap
    self._bootstrap_inner()
  File "/usr/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/databricks/python/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 521, in run
    self.wait_for_commands()
  File "/databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 593, in wait_for_commands
    command = smart_decode(self.stream.readline())[:-1]
  File "/usr/lib/python3.12/socket.py", line 707, in readinto
    return self._sock.recv_into(b)

DEBUG:ThreadMonitor:Logging Th

In [0]:
df_qtr = df_fact_airline_tickets.filter(
    (col("source_year") == "2024") &
    (col("source_quarter") == "03")
)

(
    df_qtr
    .write
    .format("delta")
    .mode("append")
    .partitionBy("source_year", "source_quarter")
    .save(GOLD_FACT_AIRLINE_TICKETS_PATH)
)

In [0]:
df_qtr = df_fact_airline_tickets.filter(
    (col("source_year") == "2024") &
    (col("source_quarter") == "04")
)

(
    df_qtr
    .write
    .format("delta")
    .mode("append")
    .partitionBy("source_year", "source_quarter")
    .save(GOLD_FACT_AIRLINE_TICKETS_PATH)
)

In [0]:
df_qtr = df_fact_airline_tickets.filter(
    (col("source_year") == "2025") &
    (col("source_quarter") == "01")
)

(
    df_qtr
    .write
    .format("delta")
    .mode("append")
    .partitionBy("source_year", "source_quarter")
    .save(GOLD_FACT_AIRLINE_TICKETS_PATH)
)

DEBUG:ThreadMonitor:Logging python thread stack frames for MainThread and py4j threads:
DEBUG:ThreadMonitor:Logging Thread-65 (run) stack frames:
  File "/usr/lib/python3.12/threading.py", line 1030, in _bootstrap
    self._bootstrap_inner()
  File "/usr/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/databricks/python/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 521, in run
    self.wait_for_commands()
  File "/databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 593, in wait_for_commands
    command = smart_decode(self.stream.readline())[:-1]
  File "/usr/lib/python3.12/socket.py", line 707, in readinto
    return self._sock.recv_into(b)

DEBUG:ThreadMonitor:Logging Th

In [0]:
df_qtr = df_fact_airline_tickets.filter(
    (col("source_year") == "2025") &
    (col("source_quarter") == "02")
)

(
    df_qtr
    .write
    .format("delta")
    .mode("append")
    .partitionBy("source_year", "source_quarter")
    .save(GOLD_FACT_AIRLINE_TICKETS_PATH)
)